In [1]:
import pandas as pd
import numpy as np
import requests
import os
import tensorflow as tf

In [2]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [3]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [4]:
ID = pd.read_csv("IDs.csv")["video_id"][0:10]
ID

0    --0bCF-iK2E
1    --14w5SOEUs
2    --40TEbZ9Is
3    --4tfbSyYDE
4    --DKkzWVh-E
5    --FmExEAsM8
6    --tbUe0JRc8
7    -024Swollbc
8    -0PZSxZuAXQ
9    -0QSEZIqVWc
Name: video_id, dtype: object

In [5]:
IDS_df = pd.DataFrame(ID)
IDS_df

,video_id
0,--0bCF-iK2E
1,--14w5SOEUs
2,--40TEbZ9Is
3,--4tfbSyYDE
4,--DKkzWVh-E
5,--FmExEAsM8
6,--tbUe0JRc8
7,-024Swollbc
8,-0PZSxZuAXQ
9,-0QSEZIqVWc


In [6]:
API_KEY = os.environ.get('API_KEY')

In [7]:
# for i in range(10):
#     file_name = f"data{i}.csv"  

#     df = pd.read_csv(file_name)
    
#     #jesus_was_a_jew_df = sentiment_score_comment(df)
    
    
    


In [8]:
df = pd.read_csv('comments/comments0_95.csv')

In [56]:
df.value_counts('video_id').keys()

Index(['-4n2JbVeJGM', '-7ehhoj_SV4', '-A1J8rzJp8E', '-72guOUhhvQ',
       '-5Rr3tf9Q6I', '-92HQA0GcI8', '-0Yxqcm0K2I', '-1jxmwecStY',
       '-177nYc5pWk', '-89ReJ8VnIM', '-0k6PPAC2SQ', '-4A3lQBM80o',
       '-90tQu_MblA', '-9fOoeMdJDI', '-2_zzwi3Q2Y', '-4ypIQAeYmY',
       '-0wz7n-h71M', '-3AikR9ZxJg', '-3uqCQ6_isc', '--0bCF-iK2E',
       '-36dal7u6BE', '-9-8gvYantk', '-0dbirUY_uk', '-43UfQZSwWc',
       '-555Hg5_Qqo', '-4vgLF7eFEQ', '-2dumMYHY3A', '--DKkzWVh-E',
       '-9DdW5VO5SA', '-6XRixsPmEs', '-8GlFQYu9O8', '-764AryBzOA',
       '-6Ildn2Dq2A', '-2exRhKYqRk', '-2X-pL06628', '-9WDEFtSU_M',
       '-3HBAvkc4a0', '-3Z7TLK22Qw', '-5q5mZbe3V8', '-ARdLKWyGfA',
       '-0pEMNPpDc4', '-7THK7BO1yA', '-0beFQnB5lY', '-0QSEZIqVWc',
       '-6Lig3HcpRg', '-0PZSxZuAXQ', '-024Swollbc', '-A3NNl0rdWs',
       '-64p7-s6yow', '-A-rx5Ka5Tk', '-1oWZTXDeeE'],
      dtype='object', name='video_id')

In [81]:
def df_cutter(df):
    IDs_list = df.value_counts('video_id').keys()
    cut_dfs = []
    for i, video in enumerate(IDs_list):
        cut_df = df[df['video_id'] == IDs_list[i]]
        cut_dfs.append(cut_df)
    return cut_dfs

In [82]:
cut_dfs = df_cutter(df)

In [86]:
cut_dfs[4]

,Unnamed: 0,comment,author,likecount,date,replies,comment_clean,video_id,weight
4451,0,"Nicole was born to be a mom, the emotions she ...",Delice Gomes,1454,2020-08-27T04:36:10Z,3,0 nicole was born to be a mom the emotions ...,-5Rr3tf9Q6I,5
4452,1,This makes me so happy. She seems more chill w...,Ruby Roberts,1008,2020-08-29T18:33:34Z,7,0 this makes me so happy she seems more chi...,-5Rr3tf9Q6I,5
4453,2,Hearing Nicole saying “my son” and hearing “Sh...,Luz Galan,71,2020-08-30T18:18:11Z,0,0 hearing nicole saying my son and hearing ...,-5Rr3tf9Q6I,4
4454,3,I'm getting emotional watching this! The love ...,Alba Talie Phillips,166,2020-08-26T16:28:12Z,1,0 im getting emotional watching this the lo...,-5Rr3tf9Q6I,5
4455,4,"This story is so incredibly amazing ,,, Brie h...",Lisa Lou,77,2020-08-28T08:44:13Z,1,0 this story is so incredibly amazing brie...,-5Rr3tf9Q6I,4
...,...,...,...,...,...,...,...,...,...
4643,192,finally the dreams of nicole to be a mother is...,Becky Carreiro,0,2020-08-27T21:23:59Z,0,0 finally the dreams of nicole to be a moth...,-5Rr3tf9Q6I,1
4644,193,The boys are technically half siblings since N...,Jennifer Grano,652,2020-08-26T15:24:44Z,39,0 the boys are technically half siblings si...,-5Rr3tf9Q6I,5
4645,194,HAHAHA so true when she mentioned about being ...,jewelz v,3,2020-08-27T01:04:53Z,0,0 hahaha so true when she mentioned about b...,-5Rr3tf9Q6I,3
4646,195,Just the most precious thing ever ❤❤❤,kelvianna epperson,0,2020-08-29T18:54:20Z,0,0 just the most precious thing ever ❤❤❤\nNa...,-5Rr3tf9Q6I,1


In [10]:
likes = df[df['likecount'] > 0]['likecount'].sort_values(ascending=True)

In [11]:
df['likecount']

0       1932
1        617
2       2583
3         57
4         66
        ... 
7709       4
7710       1
7711       0
7712      56
7713       0
Name: likecount, Length: 7714, dtype: int64

In [12]:
# likes = sorted(list(df[df['likecount'] > 0]['likecount']))

In [13]:
def making_weights(num):
    '''This function makes weights for each comment based on its like count (num)'''
    if num == 0:
        return 1
    elif num > 0 and num <= np.median(likes[:len(likes)//2]):
        return 2
    elif num > np.median(likes[:len(likes)//2]) and num <= np.median(likes):
        return 3
    elif num > np.median(likes) and num < np.median(likes[len(likes)//2:]):
        return 4
    else:
        return 5

In [14]:
df['weight'] = df['likecount'].apply(making_weights)

In [15]:
for text in df['comment']:
        # Tokenization
        tokens = tokenizer.encode_plus(text, add_special_tokens=True, padding='longest', truncation=True, max_length=512, return_tensors='tf')

        # Sentiment Prediction
        outputs = model(tokens.input_ids)
        logits = outputs.logits
        prediction = np.array(tf.nn.softmax(logits)[0])

        # Interpretation
        predicted_class = tf.argmax(logits, axis=1).numpy()[0]
        sentiment_labels = ["Negative", "Neutral", "Positive"]
        predicted_sentiment = sentiment_labels[predicted_class]

        # Print the sentiment prediction
        print(f"Text: {text}")
        print(f"Predicted Negative: {round(prediction[0]*100,2)}%")
        print(f"Predicted Neutral: {round(prediction[1]*100,2)}%")
        print(f"Predicted Positive: {round(prediction[2]*100,2)}%\n")



Text: Respect to Dortmund fans,must be sad losing him.
Predicted Negative: 31.52%
Predicted Neutral: 48.76%
Predicted Positive: 19.72%

Text: A real talent, hope he doesn't turn into another Kagawa or Mkhitaryan
Predicted Negative: 10.17%
Predicted Neutral: 65.4%
Predicted Positive: 24.43%

Text: wish him well in manchester united
Predicted Negative: 0.31%
Predicted Neutral: 11.78%
Predicted Positive: 87.91%

Text: One of the most talented players Bundesliga has ever seen. Sancho is fantastic
Predicted Negative: 0.22%
Predicted Neutral: 1.68%
Predicted Positive: 98.1%

Text: What a spell in the Bundesliga, Goodluck in the future Jadon Sancho💪💪
Predicted Negative: 0.17%
Predicted Neutral: 3.96%
Predicted Positive: 95.87%

Text: He’s a generational talent, regardless of his situation with England. He’s gonna light the prem up differently 🔥
Predicted Negative: 0.37%
Predicted Neutral: 6.25%
Predicted Positive: 93.38%

Text: The amount of opportunities he creates is ridiculous. Goal and Sh

Text: I imagine this must be similar to how United fans felt when Cristiano left for Madrid. No hard feelings, phenomenal player, only fair that he wants to play in his home country.
Predicted Negative: 8.04%
Predicted Neutral: 49.54%
Predicted Positive: 42.42%

Text: He is incredible but I think it was far too early for him to return to England. He should have stayed in Germany or moved to Spain to sharpen his skills further
Predicted Negative: 19.89%
Predicted Neutral: 48.28%
Predicted Positive: 31.83%

Text: He is one of the most talented right wingers in the bundasliga and he will be excellent for Man Utd
Predicted Negative: 0.16%
Predicted Neutral: 1.63%
Predicted Positive: 98.21%

Text: Awwwwh, what a shame to see a player of his talent leave the Bundesliga, that being said, best of luck to him at Manchester United! ❤️
Predicted Negative: 25.51%
Predicted Neutral: 36.99%
Predicted Positive: 37.5%

Text: Dortmund should be proud of their scouting/youth system bleeding talent such 

Text: I wish him all the best to Man United
Predicted Negative: 0.22%
Predicted Neutral: 6.28%
Predicted Positive: 93.5%

Text: I watched him from his first match now hes gone.Danke Jadon💛🖤💛🖤🖤💛
Predicted Negative: 29.13%
Predicted Neutral: 60.74%
Predicted Positive: 10.13%

Text: Sorry Dortmund I love Jadon big respect for making him who he is even after city releasing him/selling him
Predicted Negative: 1.93%
Predicted Neutral: 13.44%
Predicted Positive: 84.62%

Text: The best footballers have great football minds, it's like their brain reads the game differently and on another level from average players.
Predicted Negative: 1.82%
Predicted Neutral: 15.95%
Predicted Positive: 82.23%

Text: 💚Wish him be a united legend 💚
Predicted Negative: 0.3%
Predicted Neutral: 8.21%
Predicted Positive: 91.49%

Text: we will miss him
Predicted Negative: 29.61%
Predicted Neutral: 52.37%
Predicted Positive: 18.02%

Text: Wish him luck to face the great defender in EPL
Predicted Negative: 0.9%
Predicte

Text: Thank you Jadon 🖤💛🐝
Predicted Negative: 0.14%
Predicted Neutral: 3.28%
Predicted Positive: 96.58%

Text: As United fan I welcome him at Manchester
Predicted Negative: 0.19%
Predicted Neutral: 6.67%
Predicted Positive: 93.14%

Text: Sancho and Ronaldo would be like Batman and Robin
Predicted Negative: 1.02%
Predicted Neutral: 53.12%
Predicted Positive: 45.86%

Text: i dont remember the last time i've seen a daring English winger with flair. most English wingers are bland cross machines or speed merchants.
Predicted Negative: 53.78%
Predicted Neutral: 41.02%
Predicted Positive: 5.2%

Text: Now he'll play alongside RASHFORD,CAVANI,FERNANDES,POGBA💥💥
Predicted Negative: 0.3%
Predicted Neutral: 26.48%
Predicted Positive: 73.22%

Text: Still can't believe Dortmund failed to achieve smthng big inspite of having a creative playmaker (Sancho), a deadly finisher (Haaland) and a reasonably good team
Predicted Negative: 62.49%
Predicted Neutral: 30.84%
Predicted Positive: 6.67%

Text: Hope he

Text: It's only a very small exaggeration to say that your videos are the reason I signed up for an Integrated Master's of Engineering with the UK's Open University.  I'm just three months in but loving it.  Thank you for inspiring me!
Predicted Negative: 0.16%
Predicted Neutral: 0.9%
Predicted Positive: 98.94%

Text: I remember your video on Mechanically Reinforced Earth specifically because it blew me away. The cube of earth that held up your car got me hooked on your channel and I have been enjoying it ever since. 
   I appreciate that your channel is so informative about principles that operate in the background and go unappreciated by the general public, yet is so vital to our society in every day life. Thank you for sharing this with us. It is very much appreciated.
Predicted Negative: 0.16%
Predicted Neutral: 0.88%
Predicted Positive: 98.96%

Text: Years ago as a baby apprentice electrician, I worked on a job that went 6 stories down from the street for parking. They had built a

...

Text: not gonna lie that textured surface looks cool but seems like it will be a nightmare to keep clean if it attracts dust like my ps4, which is quite literally a dust magnet
Predicted Negative: 71.84%
Predicted Neutral: 22.09%
Predicted Positive: 6.07%

Text: The console is definitely big. But I love that futuristic design. The digital edition PS5 looks like the best next-gen console out of everything from Sony or Microsoft.
Predicted Negative: 0.1%
Predicted Neutral: 0.99%
Predicted Positive: 98.9%

Text: tbh i think its managable but i dont think ill be able to fit it in to the small space my ps4s in (ps im buying it soon😆)
Predicted Negative: 23.06%
Predicted Neutral: 53.87%
Predicted Positive: 23.07%

Text: I can't wait to see what the scuff gaming controller is going to look, feel and perform like!? I am so freaking excited!!!
Predicted Negative: 0.39%
Predicted Neutral: 0.78%
Predicted Positive: 98.83%

Text: Its considerably bigger then I expected. I personally wont be gettin

Text: The Ultimate Next Generation console 😍🔥🔥🔥
Predicted Negative: 0.24%
Predicted Neutral: 3.1%
Predicted Positive: 96.66%

Text: It looks so amazing. I can’t wait to get it🥺
Predicted Negative: 0.23%
Predicted Neutral: 0.62%
Predicted Positive: 99.15%

Text: 825 GB
Call of Duty: “I’ll be taking that"
Predicted Negative: 9.19%
Predicted Neutral: 82.71%
Predicted Positive: 8.1%

Text: Ali A: Notice the rip on the box.

Ali A: I hope it’s still there!
Predicted Negative: 3.33%
Predicted Neutral: 58.04%
Predicted Positive: 38.62%

Text: i am so hyped like i cant even express it! a new playstation! i love everything about it and i saw its way better that the ps4 but ofc i have to play both to confirm that but still im hyped!
Predicted Negative: 0.25%
Predicted Neutral: 0.62%
Predicted Positive: 99.13%

Text: Can’t wait for the ps5 to drop been counting down the days till it releases !!!!! ❤️❤️❤️❤️
Predicted Negative: 0.15%
Predicted Neutral: 0.67%
Predicted Positive: 99.18%

Text: Ali-A 

Text: The design is so sick
Predicted Negative: 92.63%
Predicted Neutral: 6.43%
Predicted Positive: 0.94%

Text: theRadBrad: Microsoft gave me early Xbox series x.

Ali-a: Sony gave me early PS5.

I was waiting for that kind of video.
Predicted Negative: 6.87%
Predicted Neutral: 58.49%
Predicted Positive: 34.64%

Text: Hope fully we get a ps5 SLIM (I don’t have enough space for a big console like that)
Predicted Negative: 25.06%
Predicted Neutral: 54.1%
Predicted Positive: 20.84%

Text: I can’t wait to get the ps5 💯💯
Predicted Negative: 0.23%
Predicted Neutral: 1.59%
Predicted Positive: 98.18%

Text: IM SO HYPED FOR PS5!!! I think what were going to do is sell my ps4 for the ps5 if we can but I got my ps4 around April 2016 so it’s been a while
Predicted Negative: 0.14%
Predicted Neutral: 1.16%
Predicted Positive: 98.7%

Text: Download Monster Legends now and get this special reward for free until September 26th! 
https://monsterlegends.onelink.me/QSYm/Silver
Predicted Negative: 0.21%
P

Text: Let’s talk about how at the 20 second diamond mining Timelapse he synced the noise of mining diamonds to the song
Predicted Negative: 5.26%
Predicted Neutral: 87.79%
Predicted Positive: 6.94%

Text: Sheeesh the ending was so good and seriously both these 200 days and the next 100 days will be the best videos I have ever seen.
Also needed to add this, the amount of effort required to make a video this interesting is insane, I really hope you get even more views than before
Predicted Negative: 0.27%
Predicted Neutral: 1.28%
Predicted Positive: 98.46%

Text: THE  EFFORT IS  INSANE! RESPECT
Predicted Negative: 3.24%
Predicted Neutral: 16.41%
Predicted Positive: 80.34%

Text: “Definitely not the best builder ever” Him building something better than I ever have
Predicted Negative: 15.86%
Predicted Neutral: 39.42%
Predicted Positive: 44.72%

Text: So much good videos bro keep up the good work and you'll hit your goal in no time
Predicted Negative: 0.16%
Predicted Neutral: 1.51%
Predicte

Text: Watched this like a billion times by now cuz it is so addicting XD
Predicted Negative: 0.25%
Predicted Neutral: 1.98%
Predicted Positive: 97.76%

Text: silver:has the most powerful shapeshifts to use against the obsidilith
also silver:mmmm monkey
Predicted Negative: 4.45%
Predicted Neutral: 65.34%
Predicted Positive: 30.21%

Text: Silver: I’m not the best at building
Me: I can’t even build something that good in creative
Predicted Negative: 62.2%
Predicted Neutral: 30.61%
Predicted Positive: 7.19%

Text: I like how Preston played the lava wizard, it suits him.
Predicted Negative: 0.41%
Predicted Neutral: 6.71%
Predicted Positive: 92.88%

Text: You can imagine that if you support this dude, he will give you a actuall mod of shapeshifter
Predicted Negative: 36.17%
Predicted Neutral: 55.42%
Predicted Positive: 8.41%

Text: love how when he's selecting his 7 shapeshifts theirs pokemon music in the background
Predicted Negative: 0.45%
Predicted Neutral: 15.64%
Predicted Positive: 83.9

In [16]:
prediction[0]

0.13986312

In [17]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification


In [18]:
def sentiment_score_comment(df):

    '''This function predicts the sentiment score of each youtube video!'''
    
    model_name = "cardiffnlp/twitter-roberta-base-sentiment"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

    # Lists to store the sentiment analysis results
    sentiment_list = []
    negative_list = []
    neutral_list = []
    positive_list = []
    scalar_value_list = []
    weighted_SV = []
    weight = list(df['likecount'].apply(making_weights))

    # Iterate over the comments in the DataFrame
    for i, text in enumerate(df['comment']):
        
        # Tokenization, Sentiment Prediction, and Interpretation
        tokens = tokenizer.encode_plus(text, add_special_tokens=True, padding='longest', truncation=True, max_length=512, return_tensors='tf')
        outputs = model(tokens.input_ids)
        logits = outputs.logits
        prediction = np.array(tf.nn.softmax(logits)[0])
        predicted_class = tf.argmax(logits, axis=1).numpy()[0]
        sentiment_labels = ["Negative", "Neutral", "Positive"]
        predicted_sentiment = sentiment_labels[predicted_class]

        # Append the sentiment analysis results to the respective lists
        sentiment_list.append(predicted_sentiment)
        negative_list.append(round(prediction[0]*100, 2))
        neutral_list.append(round(prediction[1]*100, 2))
        positive_list.append(round(prediction[2]*100, 2))
        scalar_value_val = round((prediction[0])*-1+(prediction[2]*1),2)
        scalar_value_list.append(scalar_value_val)
        weighted_SV.append(df['weight'].iloc[i] * scalar_value_val)


    # Create a new DataFrame with the sentiment analysis results
    results_df = pd.DataFrame({
        'Comment': df['comment'],
        'Sentiment': sentiment_list,
        'Negative (%)': negative_list,
        'Neutral (%)': neutral_list,
        'Positive (%)': positive_list,
        'Scaler_value': scalar_value_list,
        'weighted_SV': weighted_SV,
        'weight': weight
    })

    # Return the new DataFrame
    return results_df


In [19]:
comment = 'I really love le wagon and it is so nice and warm'

In [88]:
for df in cut_dfs:
    results_list = []
    results = sentiment_score_comment(df)
    results_list.append(results)
results_list

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the mode

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the mode

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the mode

SyntaxError: 'return' outside function (4132658713.py, line 5)

In [90]:
len(results_list)

1

In [21]:
final_df = pd.DataFrame(df)

In [22]:
# IDS_df['positivity_score'] = np.nan

In [24]:
results

,Comment,Sentiment,Negative (%),Neutral (%),Positive (%),Scaler_value,weighted_SV,weight
0,"Respect to Dortmund fans,must be sad losing him.",Neutral,31.52,48.76,19.72,-0.12,-0.60,5
1,"A real talent, hope he doesn't turn into anoth...",Neutral,10.17,65.40,24.43,0.14,0.70,5
2,wish him well in manchester united,Positive,0.31,11.78,87.91,0.88,4.40,5
3,One of the most talented players Bundesliga ha...,Positive,0.22,1.68,98.10,0.98,3.92,4
4,"What a spell in the Bundesliga, Goodluck in th...",Positive,0.17,3.96,95.87,0.96,3.84,4
...,...,...,...,...,...,...,...,...
7709,Kinda miss our skyblock content but its good t...,Positive,0.18,1.24,98.58,0.98,2.94,3
7710,Silver: this is a hardcore world anyway\nSilve...,Negative,55.05,38.95,6.00,-0.49,-0.98,2
7711,He gave the ocean wizard very quickly by sayin...,Neutral,1.91,74.14,23.95,0.22,0.22,1
7712,Hi silver! I wanted to ask you that when are y...,Positive,0.63,3.94,95.43,0.95,3.80,4


In [91]:
# for i in range(10):
#     file_name = f"data{i}.csv"  

#     df = pd.read_csv(file_name)
    
#     df['weight'] = df['likecount'].apply(making_weights)
    
#     peppa_pig_df = sentiment_score_comment(df)
    
#     positivity_score = peppa_pig_df['weighted_SV'].mean()
    
#     IDS_df.iloc[i]['positivity_score'] = positivity_score
    
    
    


In [ ]:
results_df['weighted_SV'].mean()

In [ ]:
results_df

In [ ]:
results = sentiment_score_comment(df)

In [ ]:
results

In [ ]:
results_df = pd.DataFrame({
    'Comment': df['comment'],
    'Sentiment': sentiment_list,
    'Negative (%)': negative_list,
    'Neutral (%)': neutral_list,
    'Positive (%)': positive_list
})

# Print the new DataFrame
print(results_df)

In [ ]:
import pandas as pd
import numpy as np

# Lists to store the sentiment analysis results
sentiment_list = []
negative_list = []
neutral_list = []
positive_list = []

# Iterate over the comments in the DataFrame
for text in df['comment']:
    # Tokenization, Sentiment Prediction, and Interpretation
    tokens = tokenizer.encode_plus(text, add_special_tokens=True, padding='longest', truncation=True, max_length=512, return_tensors='tf')
    outputs = model(tokens.input_ids)
    logits = outputs.logits
    prediction = np.array(tf.nn.softmax(logits)[0])
    predicted_class = tf.argmax(logits, axis=1).numpy()[0]
    sentiment_labels = ["Negative", "Neutral", "Positive"]
    predicted_sentiment = sentiment_labels[predicted_class]

    # Append the sentiment analysis results to the respective lists
    sentiment_list.append(predicted_sentiment)
    negative_list.append(round(prediction[0]*100, 2))
    neutral_list.append(round(prediction[1]*100, 2))
    positive_list.append(round(prediction[2]*100, 2))

# Create a new DataFrame with the sentiment analysis results
results_df = pd.DataFrame({
    'Comment': df['comment'],
    'Sentiment': sentiment_list,
    'Negative (%)': negative_list,
    'Neutral (%)': neutral_list,
    'Positive (%)': positive_list
})

# Print the new DataFrame
print(results_df)


In [ ]:
results_df